In [1]:
import os 
import torch

In [2]:
print(os.getenv("CONDA_DEFAULT_ENV"))

vid_env


In [3]:
from transformers import AutoConfig, AutoModel
from transformers import AutoTokenizer, AutoProcessor

In [4]:
from PIL import Image
from decord import VideoReader, cpu

### Model Loading

In [5]:
model_path = 'mPLUG/mPLUG-Owl3-7B-240728'

In [6]:
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, torch_dtype=torch.half, trust_remote_code=True, config=config)
tokenizer = AutoTokenizer.from_pretrained(model_path)
processor = model.init_processor(tokenizer)

/home/aritrad/miniconda3/envs/vid_env/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


use flash_attn rotary


/home/aritrad/miniconda3/envs/vid_env/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
_ = model.eval().cuda()

### Prepare Chat Messages

In [8]:
messages = [
    {
        "role": "user", 
        "content": """<|video|> Describe this video."""
    },
    {
        "role": "assistant", 
        "content": ""
    }
]

### Load Test Video

In [9]:
videos = ['/home/aritrad/MSR-Project/samples/4min-video.mp4']
MAX_NUM_FRAMES=16

In [10]:
def encode_video(video_path):
    def uniform_sample(l, n):
        gap = len(l) / n
        idxs = [int(i * gap + gap / 2) for i in range(n)]
        return [l[i] for i in idxs]

    vr = VideoReader(video_path, ctx=cpu(0))
    sample_fps = round(vr.get_avg_fps() / 1)  # FPS
    frame_idx = [i for i in range(0, len(vr), sample_fps)]
    if len(frame_idx) > MAX_NUM_FRAMES:
        frame_idx = uniform_sample(frame_idx, MAX_NUM_FRAMES)
    frames = vr.get_batch(frame_idx).asnumpy()
    frames = [Image.fromarray(v.astype('uint8')) for v in frames]
    print('num frames:', len(frames))
    return frames

In [11]:
video_frames = [encode_video(_) for _ in videos]
inputs = processor(messages, images=None, videos=video_frames)

num frames: 16


In [12]:
inputs.to('cuda')
inputs.update({
    'tokenizer': tokenizer,
    'max_new_tokens':256,
    'decode_text':True,
})

In [13]:
g = model.generate(**inputs)
print(g)

['The video shows a man in a white shirt giving a lecture on electrical engineering. He is standing behind a podium and using a projector to display slides. The video also includes a slide with the course outline and a slide with the course objectives.']
